In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
#!pip install dask[dataframe]
#!pip install pandas --upgrade
# We have to do this as dask is bad at guessing the types of objects for some reason
# Also, if you are having trouble with running the dataframe and continuously get 
# Fatal Python error: GC object already tracked, try to update pandas
data_types = {
    'ID':  np.int64,
    'Case Number':  object,
    'Date': object,
    'Block': object,
    'IUCR': object,
    'Primary Type': object,
    'Description':  object,
    'Location Description': object,
    'Arrest':  bool,
    'Domestic': bool,
    'Beat': np.int64,
    'District': np.float64,
    'Ward': np.float64,
    'Community Area': np.float64,
    'FBI Code': object,
    'X Coordinate': np.float64,
    'Y Coordinate': np.float64,
    'Year': np.int64,
    'Updated On': object,
    'Latitude': np.float64,
    'Longitude': np.float64,
    'Location': object
}
df = pd.read_csv('crimes.csv')

# get subset of crimes -- with over ~100 entries, the geolocator.geocode() call times out
crimes_sub  = df[:50]
crimes_sub.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,7,43,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
1,10446329,HZ185174,3/12/2016 23:50:00,034XX N HALSTED ST,460,BATTERY,SIMPLE,STREET,False,False,...,44,6,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
2,10446859,HZ185472,3/12/2016 23:50:00,006XX S CLARK ST,460,BATTERY,SIMPLE,RESIDENCE,False,False,...,2,32,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
3,10446323,HZ185185,3/12/2016 23:50:00,078XX S ABERDEEN ST,497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,True,True,...,17,71,04B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN
4,10446385,HZ185156,3/12/2016 23:45:00,068XX S JUSTINE ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,17,67,3,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN


In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

# Gets full address using block address with "Chicago" appended
# Extracts zip code from full address
#
def getZip(row):
    location = geolocator.geocode(row['Block'] + " Chicago")
    if location is None: 
        return 0
    address = location.address.replace(',', "")
    return next(iter([int(s) for s in address.split() if s.isdigit()]), 0)

# Add zip code column with zip code of crime
crimes_sub['Zip Code'] = crimes_sub.apply(getZip, axis=1, reduce=False)
crimes_sub.head()

/Users/kelseyball/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Zip Code
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,43,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN,60617
1,10446329,HZ185174,3/12/2016 23:50:00,034XX N HALSTED ST,460,BATTERY,SIMPLE,STREET,False,False,...,6,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN,60642
2,10446859,HZ185472,3/12/2016 23:50:00,006XX S CLARK ST,460,BATTERY,SIMPLE,RESIDENCE,False,False,...,32,08B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN,60605
3,10446323,HZ185185,3/12/2016 23:50:00,078XX S ABERDEEN ST,497,BATTERY,AGGRAVATED DOMESTIC BATTERY: OTHER DANG WEAPON,APARTMENT,True,True,...,71,04B,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN,60608
4,10446385,HZ185156,3/12/2016 23:45:00,068XX S JUSTINE ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,67,3,NaN,NaN,2016,3/19/2016 15:45:00,NaN,NaN,NaN,60609


In [3]:
poverty = pd.read_csv('poverty.csv')
poverty.head()


,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [4]:
# Socioeconomic stats by community area 
# https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2
#
poverty = pd.read_csv('poverty.csv')
poverty.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39
1,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46
2,3,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20
3,4,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17
4,5,North Center,0.3,7.5,5.2,4.5,26.2,57123,6


In [5]:
# Mapping of community area numbers to zip codes
# https://docs.google.com/spreadsheets/d/1YzfGnfeiLdYtbN8IoOctPZPJRCAms_gW6g-OTvkGAZU/edit#gid=0
#

zip_codes = pd.read_csv('zipcodes.csv')
zip_codes.head()

,Community Area Number,Zip Code
0,1,60626
1,1,60645
2,1,60660
3,2,60626
4,2,60645


In [6]:
# Merge socioeconomic stats with zip codes
poverty_with_zip = pd.merge(poverty, zip_codes, how='right', on='Community Area Number')
poverty_with_zip.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX,Zip Code
0,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,60626
1,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,60645
2,1,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39,60660
3,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,60626
4,2,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46,60645


In [7]:
# Merge crimes with socioeconomic data for relevant zip code
crimes_with_poverty = pd.merge(crimes_sub, poverty_with_zip, how='left', on='Zip Code')
crimes_with_poverty.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Zip Code,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,60617,45,Avalon Park,1.4,17.2,21.1,10.6,39.3,24454,41
1,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,60617,46,South Chicago,4.7,29.8,19.7,26.6,41.1,16579,75
2,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,60617,48,Calumet Heights,2.1,11.5,20.0,11.0,44.0,28887,38
3,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,60617,51,South Deering,4.0,29.2,16.3,21.0,39.5,14685,65
4,10446260,HZ185168,3/12/2016 23:50:00,078XX S YATES BLVD,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,60617,52,East Side,6.8,19.2,12.1,31.9,42.8,17104,64
